Imports

In [14]:
# Systèmes de fichiers et manipulations diverses
import os
import numpy as np

# PyTorch et modules associés
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.models as models
from torchvision import transforms
from torch.cuda.amp import GradScaler, autocast

# Gestion des fichiers .h5
import h5py

# Division des ensembles (train/val/test) et calcul des métriques
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, classification_report


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Transfo def

In [17]:
# Définir les transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionner l'image à 224x224
    transforms.ToTensor(),  # Convertir l'image en format [C, H, W] et normaliser entre 0 et 1
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalisation pour ResNet
])

classe personnalisé pour le dataset

In [ ]:
class DogDataset(Dataset):
    def __init__(self, h5_file_path, transform=None):
        self.h5_file = h5py.File(h5_file_path, 'r')
        self.images = self.h5_file['images']
        self.labels = self.h5_file['labels'] 
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = torch.tensor(self.images[idx], dtype=torch.float16)
        label = torch.tensor(self.labels[idx], dtype=torch.long)

        if len(image.shape) == 3 and image.shape[2] == 3:
            image = image.permute(2, 0, 1)

        if self.transform:
            # Convertir temporairement l'image en float32 pour les transformations
            image = image.float()
            image = self.transform(image)
            image = image.half() 

        return image, label

classe de Early stopping

In [16]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_loss = float('inf')
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

Création des dataloader

In [ ]:
# Charger le dataset
h5_file_path = 'D:/ApprentissageMachineTPFinal/data/dog_dataset_no_aug.h5'
dataset = DogDataset(h5_file_path)

# Diviser en ensembles train/val/test
indices = list(range(len(dataset_subset)))
train_indices, temp_indices = train_test_split(indices, test_size=0.3, random_state=42)
val_indices, test_indices = train_test_split(temp_indices, test_size=0.5, random_state=42)

# Créer les DataLoaders
train_loader = DataLoader(Subset(dataset_subset, train_indices), batch_size=64, shuffle=True)
val_loader = DataLoader(Subset(dataset_subset, val_indices), batch_size=64, shuffle=False)
test_loader = DataLoader(Subset(dataset_subset, test_indices), batch_size=64, shuffle=False)

# Filtrer uniquement les données des 3 classes (par exemple 0, 1, 2)
class_indices = [i for i, label in enumerate(dataset.labels) if label in [0, 1, 2]]
dataset_subset = Subset(dataset, class_indices)

Charger le modèle ResNet50 pré-entraîné

In [20]:
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 3)  # 3 classes
model = model.to(device)

Fonction de perte et optimiseur, scheduler pour le learning rate et initialisation de l'early stopping

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3)

early_stopping = EarlyStopping(patience=5)

scaler = torch.cuda.amp.GradScaler()

C:\Users\nhousset\AppData\Local\Temp\ipykernel_19340\967691801.py:7: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


Mesures de performances

In [22]:
def evaluate_model(model, data_loader, device):
    model.eval()  # Mettez le modèle en mode évaluation
    all_labels = []
    all_preds = []

    with torch.no_grad():  # Désactivez le calcul des gradients
        for images, labels in data_loader:
            images = images.to(device, dtype=torch.float16)  # Convertir en float16
            labels = labels.to(device)

            outputs = model(images)  # Passer les images au modèle
            _, preds = torch.max(outputs, 1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    
    # Calcul des métriques
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    report = classification_report(all_labels, all_preds)
    
    return accuracy, f1, recall, report

Vérification des dimensions des images avant d'entrer dans le modèle

In [23]:
for images, labels in train_loader:
    images = images.to(device)
    labels = labels.to(device)

Boucle d’entraînement

In [24]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device, dtype=torch.float16)
        labels = labels.to(device)

        # Convertir en float32 avant de passer dans le modèle
        images = images.to(torch.float32)

        # Passage dans le modèle
        outputs = model(images)

        # Calcul de la perte
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Mettez à jour la perte
        running_loss += loss.item()
    
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


C:\Users\nhousset\AppData\Local\Temp\ipykernel_19340\2414848011.py:25: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch [1/10], Loss: 0.5928
Epoch [2/10], Loss: 1.4652
Epoch [3/10], Loss: 1.2395
Epoch [4/10], Loss: 1.1685
Epoch [5/10], Loss: 0.7131
Epoch [6/10], Loss: 1.2097
Epoch [7/10], Loss: 0.8236
Epoch [8/10], Loss: 0.3320
Epoch [9/10], Loss: 0.1282
Epoch [10/10], Loss: 0.0956


Early Stopping (marche pas, break caput)

In [ ]:
#early_stopping(val_loss)
#if early_stopping.early_stop:
    #print("Early stopping triggered.")
    #break

Validation

In [ ]:
  # Validation
val_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device, dtype=torch.float16), labels.to(device)
        with torch.cuda.amp.autocast():
            
            images = images.to(torch.float32)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
        val_loss += loss.item()

val_loss /= len(val_loader)
print(f"Validation Loss: {val_loss:.4f}")\

#Scheduler 
scheduler.step(val_loss)

# Sauvegarde du modèle
torch.save(model.state_dict(), 'resnet50_best_model.pth')

C:\Users\nhousset\AppData\Local\Temp\ipykernel_19340\117625145.py:9: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Validation Loss: 0.9093


Validation finale


In [ ]:
model.eval()
val_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device, dtype=torch.float16)  # Charger en float16
        labels = labels.to(device)
        
        model = model.half()  # Charger le modèle en float16
        outputs = model(images)  # Passer directement les images
        loss = criterion(outputs.float(), labels)  # Convertir les sorties en float32 pour la perte

        val_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_acc = 100 * correct / total
print(f"Validation Loss: {val_loss/len(val_loader):.4f}, Validation Accuracy: {val_acc:.2f}%")

# Évaluation finale sur l'ensemble de test
accuracy, f1, recall, report = evaluate_model(model, test_loader, device)
print("Test Results:")
print(f"Accuracy: {accuracy:.2f}, F1-Score: {f1:.2f}, Recall: {recall:.2f}")
print("Classification Report:\n", report)


Validation Loss: 0.3886, Validation Accuracy: 91.43%
Test Results:
Accuracy: 0.93, F1-Score: 0.93, Recall: 0.93
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.97      0.94        35
           1       0.95      0.95      0.95        21
           2       0.92      0.79      0.85        14

    accuracy                           0.93        70
   macro avg       0.93      0.90      0.91        70
weighted avg       0.93      0.93      0.93        70

